<a href="https://colab.research.google.com/github/herysedra/ady_cov/blob/master/scr/fitaovana/Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import matplotlib
import pickle

"""
Mahasedra comments:
Here we will trace most of treatments done on the data 'confirmed_cases'
"""

#cco_cases_mdg = pd.read_csv("/content/drive/My Drive/data/cco mdg.csv")


csse_confirmed_cases_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
csse_confirmed_cases = pd.read_csv(csse_confirmed_cases_url, sep=',')

deaths_url =  'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
deaths = pd.read_csv(deaths_url, sep=',')

csse_recovered_cases_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
csse_recovered_cases = pd.read_csv(csse_recovered_cases_url, sep=',')

who_conf_and_death_cases_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/who_covid_19_situation_reports/who_covid_19_sit_rep_time_series/who_covid_19_sit_rep_time_series.csv'
who_conf_and_death_cases = pd.read_csv(who_conf_and_death_cases_url, sep=',')


who_cases_mdg = pd.read_csv("/content/drive/My Drive/data/who mdg.csv")

cco_cases_mdg = pd.read_csv("/content/drive/My Drive/data/cco mdg.csv")


tilyvoa = pd.read_csv("/content/drive/My Drive/data/tilyvoa.csv")

mdg_databe = pd.read_csv("/content/drive/My Drive/data/databe mdg.csv")

#tilyvoa.set_index('Daty', inplace=True)

tilyvoafar = tilyvoa.groupby('Faritra')
tilyvoamdg = tilyvoafar.get_group('Madagasikara')

tilyvoamdg.set_index('Daty', inplace=True)

tilyvoamdg.index = pd.to_datetime(tilyvoamdg.index)

tilyvoatana = tilyvoafar.get_group('Antananarivo')
tilyvoatana.set_index('Daty', inplace=True)

tilyvoatana.index = pd.to_datetime(tilyvoatana.index)

tilyvoatoa = tilyvoafar.get_group('Toamasina')
tilyvoatoa.set_index('Daty', inplace=True)

tilyvoatoa.index = pd.to_datetime(tilyvoatoa.index)

who_cases_mdg.set_index('Date', inplace=True)

cco_cases_mdg.set_index('Date', inplace=True)

path_to_save = '/content/drive/My Drive/sary/'
path_data = '/content/drive/My Drive/data/'



In [0]:
import pymc3 as pm
import theano.tensor as tt
import theano
import datetime
import time

date_data_begin = datetime.date(2020,3,1)
date_data_end = datetime.date(2020,3,15)
num_days_to_predict = 28


diff_data_sim = 16 # should be significantly larger than the expected delay, in 
                   # order to always fit the same number of data points.


date_begin_sim = date_data_begin - datetime.timedelta(days = diff_data_sim)

format_date = lambda date_py: '{}/{}/{}'.format(date_py.month, date_py.day,
                                                 str(date_py.year)[2:4])
date_formatted_begin = format_date(date_data_begin)
date_formatted_end = format_date(date_data_end)


print('date_data_begin is:',date_data_begin, sep=" ")
print('date_formatted_begin', date_formatted_begin, sep=" ")



date_data_begin is: 2020-03-01
date_formatted_begin 3/1/20


In [0]:
csse_confirmed_cases.info()
csse_confirmed_cases

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 100 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Province/State  82 non-null     object 
 1   Country/Region  264 non-null    object 
 2   Lat             264 non-null    float64
 3   Long            264 non-null    float64
 4   1/22/20         264 non-null    int64  
 5   1/23/20         264 non-null    int64  
 6   1/24/20         264 non-null    int64  
 7   1/25/20         264 non-null    int64  
 8   1/26/20         264 non-null    int64  
 9   1/27/20         264 non-null    int64  
 10  1/28/20         264 non-null    int64  
 11  1/29/20         264 non-null    int64  
 12  1/30/20         264 non-null    int64  
 13  1/31/20         264 non-null    int64  
 14  2/1/20          264 non-null    int64  
 15  2/2/20          264 non-null    int64  
 16  2/3/20          264 non-null    int64  
 17  2/4/20          264 non-null    in

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20
0,NaN,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,...,22,22,24,24,40,40,74,84,94,110,110,120,170,174,237,273,281,299,349,367,423,444,484,521,555,607,665,714,784,840,906,933,996,1026,1092,1176,1279,1351,1463,1531
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,59,64,70,76,89,104,123,146,174,186,197,212,223,243,259,277,304,333,361,377,383,400,409,416,433,446,467,475,494,518,539,548,562,584,609,634,663,678,712,726
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,74,87,90,139,201,230,264,302,367,409,454,511,584,716,847,986,1171,1251,1320,1423,1468,1572,1666,1761,1825,1914,1983,2070,2160,2268,2418,2534,2629,2718,2811,2910,3007,3127,3256,3382
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,39,53,75,88,113,133,164,188,224,267,308,334,370,376,390,428,439,466,501,525,545,564,583,601,601,638,646,659,673,673,696,704,713,717,717,723,723,731,738,738
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,2,2,3,3,3,4,4,5,7,7,7,8,8,8,10,14,16,17,19,19,19,19,19,19,19,19,19,19,24,24,24,24,25,25,25,25,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Saint Pierre and Miquelon,France,46.885200,-56.315900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
260,NaN,South Sudan,6.877000,31.307000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,2,3,4,4,4,4,4,4,4,4,4,4,4,4,4,5,5,5,6
261,NaN,Western Sahara,24.215500,-12.885800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,4,4,4,4,4,4,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
262,NaN,Sao Tome and Principe,0.186360,6.613081,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4


In [0]:
csse_confirmed_cases.loc[csse_confirmed_cases["Country/Region"] == "Germany"]

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20
120,NaN,Germany,51.0,9.0,0,0,0,0,0,1,4,4,4,5,8,10,12,12,12,12,13,13,14,14,16,16,16,16,16,16,16,16,16,16,16,16,16,16,17,27,...,12327,15320,19848,22213,24873,29056,32986,37323,43938,50871,57695,62095,66885,71808,77872,84794,91159,96092,100123,103374,107663,113296,118181,122171,124908,127854,130072,131359,134753,137698,141397,143342,145184,147065,148291,150648,153129,154999,156513,157770


In [0]:
cases_obs =  np.array(
    csse_confirmed_cases.loc[csse_confirmed_cases["Country/Region"] == "Germany", 
                        date_formatted_begin:date_formatted_end])[0]



print('Germany csse 3/1/20 to 3/15/20:', cases_obs, sep=" ")


print('len_(cases_obs) is', len(cases_obs), sep=" ")
type(cases_obs)
cases_obs.shape
#From the output, cases_obs is made of 15

Germany csse 3/1/20 to 3/15/20: [ 130  159  196  262  482  670  799 1040 1176 1457 1908 2078 3675 4585
 5795]
len_(cases_obs) is 15


(15,)

In [0]:
cases0_obs =  np.array(
    csse_confirmed_cases.loc[csse_confirmed_cases["Country/Region"] == "Germany", 
                        date_formatted_begin:date_formatted_end])

# Notice the absence of [0] at the end compared to cases_obs.

print(cases0_obs)
type(cases0_obs)
cases0_obs.shape
#From the output, cases0_obs is made of 1 row and 15 columns 
# The action of [0] is to pick the index 0.

In [0]:
date_data_begin = datetime.date(2020,3,1)
date_data_end = datetime.date(2020,3,15)

format_date = lambda date_py: '{}/{}/{}'.format(date_py.month, date_py.day,
                                                 str(date_py.year))

date_formatted_begin = format_date(date_data_begin)
date_formatted_end = format_date(date_data_end)

"""
cases_jw_obs =  np.array(who_conf_and_death_cases.loc[(who_conf_and_death_cases['Province/States'] == "Confirmed") & (who_conf_and_death_cases["Country/Region"] == "Germany"), 
                        date_formatted_begin:date_formatted_end])
"""
cases_jw_obs =  np.array(who_conf_and_death_cases.loc[who_conf_and_death_cases["Country/Region"] == "Germany", 
                        date_formatted_begin:date_formatted_end])[0]

print('Germany jwho 3/1/20 to 3/15/20:', cases_jw_obs)

Germany jwho 3/1/20 to 3/15/20: [  57.  129.  157.  196.  262.  534.  639.  795. 1112. 1139. 1296. 1567.
 2369. 3062. 3795.]


In [0]:
who_conf_and_death_cases.info()
who_conf_and_death_cases

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259 entries, 0 to 258
Data columns (total 77 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Province/States  53 non-null     object 
 1   Country/Region   247 non-null    object 
 2   WHO region       256 non-null    object 
 3   1/21/2020        11 non-null     float64
 4   1/22/2020        21 non-null     float64
 5   1/23/2020        25 non-null     float64
 6   1/24/2020        27 non-null     float64
 7   1/25/2020        16 non-null     float64
 8   1/26/2020        17 non-null     float64
 9   1/27/2020        19 non-null     float64
 10  1/28/2020        22 non-null     float64
 11  1/29/2020        23 non-null     float64
 12  1/30/2020        26 non-null     float64
 13  1/31/2020        27 non-null     float64
 14  2/1/2020         62 non-null     float64
 15  2/2/2020         62 non-null     float64
 16  2/3/2020         62 non-null     float64
 17  2/4/2020        

,Province/States,Country/Region,WHO region,1/21/2020,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,2/1/2020,2/2/2020,2/3/2020,2/4/2020,2/5/2020,2/6/2020,2/7/2020,2/8/2020,2/9/2020,2/10/2020,2/11/2020,2/12/2020,2/13/2020,2/14/2020,2/15/2020,2/16/2020,2/17/2020,2/18/2020,2/19/2020,2/20/2020,2/21/2020,2/22/2020,2/23/2020,2/24/2020,2/25/2020,2/26/2020,2/27/2020,2/28/2020,2/29/2020,3/1/2020,3/2/2020,3/3/2020,3/4/2020,3/5/2020,3/6/2020,3/7/2020,3/8/2020,3/9/2020,3/10/2020,3/11/2020,3/12/2020,3/13/2020,3/14/2020,3/15/2020,3/16/2020,3/17/2020,3/18/2020,3/19/2020,3/20/2020,3/21/2020,3/22/2020,3/23/2020,3/24/2020,3/25/2020,3/26/2020,3/27/2020,3/28/2020,3/29/2020,3/30/2020,3/31/2020,4/1/2020,4/2/2020,4/3/2020
0,Confirmed,Globally,NaN,282.0,314.0,581.0,846.0,1320.0,2014.0,2798.0,4593.0,6065.0,7818.0,9826.0,11953.0,14557.0,17391.0,20630.0,24554.0,28276.0,31481.0,34886.0,37558.0,40554.0,43103.0,45171.0,46997.0,49053.0,50580.0,51857.0,71429.0,73332.0,75204.0,75748.0,76769.0,77794.0,78811.0,79331.0,80239.0,81109.0,82294.0,83652.0,85403.0,87137.0,88948.0,90870.0,93091.0,95324.0,98192.0,101927.0,105586.0,109577.0,113702.0,118319.0,125048.0,132758.0,142539.0,153517.0,167515.0,179111.0,191127.0,209839.0,234073.0,266073.0,292142.0,332930.0,372755.0,413467.0,462684.0,509164.0,571659.0,634813.0,693282.0,750890.0,823626.0,896450.0,972303.0
1,Deaths,Globally,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7426.0,7807.0,8778.0,9840.0,11184.0,12783.0,14509.0,16231.0,18433.0,20834.0,23335.0,26493.0,29891.0,33106.0,36405.0,40598.0,45525.0,50321.0
2,Confirmed,China,Western Pacific Region,278.0,309.0,571.0,830.0,1297.0,1985.0,2741.0,4537.0,5997.0,7736.0,9720.0,11821.0,14411.0,17238.0,20471.0,24363.0,28060.0,31211.0,34598.0,37251.0,40235.0,42708.0,44730.0,46550.0,48548.0,50054.0,51174.0,70635.0,72528.0,74280.0,74675.0,75569.0,76392.0,77042.0,77262.0,77780.0,78191.0,78630.0,78961.0,79394.0,79968.0,80174.0,80304.0,80422.0,80565.0,80711.0,80813.0,80859.0,80904.0,80924.0,80955.0,80981.0,80991.0,81021.0,81048.0,81077.0,81116.0,81116.0,81174.0,81300.0,81416.0,81498.0,81601.0,81747.0,81848.0,81961.0,82078.0,82230.0,82341.0,82447.0,82545.0,82631.0,82724.0,82802.0
3,Confirmed,Outside of China,NaN,4.0,5.0,10.0,16.0,23.0,29.0,57.0,56.0,68.0,82.0,106.0,132.0,146.0,153.0,159.0,191.0,216.0,270.0,288.0,307.0,319.0,395.0,441.0,447.0,505.0,526.0,683.0,794.0,804.0,924.0,1073.0,1200.0,1402.0,1769.0,2069.0,2459.0,2918.0,3664.0,4691.0,6009.0,7169.0,8774.0,10566.0,12669.0,14759.0,17481.0,21110.0,24727.0,28673.0,32778.0,37364.0,44067.0,51767.0,61518.0,72469.0,86438.0,97995.0,110011.0,128665.0,152773.0,184657.0,210644.0,251329.0,291008.0,331619.0,380723.0,427086.0,489429.0,552472.0,610835.0,668345.0,740995.0,813726.0,889501.0
4,Deaths,China,Western Pacific Region,NaN,NaN,NaN,NaN,NaN,NaN,80.0,106.0,132.0,170.0,213.0,259.0,304.0,361.0,425.0,491.0,564.0,637.0,723.0,812.0,909.0,1017.0,1114.0,1260.0,1381.0,1524.0,1666.0,1772.0,1870.0,2006.0,2121.0,2239.0,2348.0,2445.0,2595.0,2666.0,2718.0,2747.0,2791.0,2838.0,2873.0,2915.0,2946.0,2984.0,3015.0,3045.0,3073.0,3100.0,3123.0,3140.0,3162.0,3173.0,4955.0,3194.0,3204.0,3218.0,3231.0,3231.0,3242.0,3253.0,3261.0,3267.0,3276.0,3283.0,3287.0,3293.0,3298.0,3301.0,3306.0,3310.0,3314.0,3321.0,3327.0,3331.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,Deaths,NaN,Eastern Mediterranean Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [0]:
who_conf_and_death_cases[0:12]

,Province/States,Country/Region,WHO region,1/21/2020,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,2/1/2020,2/2/2020,2/3/2020,2/4/2020,2/5/2020,2/6/2020,2/7/2020,2/8/2020,2/9/2020,2/10/2020,2/11/2020,2/12/2020,2/13/2020,2/14/2020,2/15/2020,2/16/2020,2/17/2020,2/18/2020,2/19/2020,2/20/2020,2/21/2020,2/22/2020,2/23/2020,2/24/2020,2/25/2020,2/26/2020,2/27/2020,2/28/2020,2/29/2020,3/1/2020,3/2/2020,3/3/2020,3/4/2020,3/5/2020,3/6/2020,3/7/2020,3/8/2020,3/9/2020,3/10/2020,3/11/2020,3/12/2020,3/13/2020,3/14/2020,3/15/2020,3/16/2020,3/17/2020,3/18/2020,3/19/2020,3/20/2020,3/21/2020,3/22/2020,3/23/2020,3/24/2020,3/25/2020,3/26/2020,3/27/2020,3/28/2020,3/29/2020,3/30/2020,3/31/2020,4/1/2020,4/2/2020,4/3/2020
0,Confirmed,Globally,NaN,282.0,314.0,581.0,846.0,1320.0,2014.0,2798.0,4593.0,6065.0,7818.0,9826.0,11953.0,14557.0,17391.0,20630.0,24554.0,28276.0,31481.0,34886.0,37558.0,40554.0,43103.0,45171.0,46997.0,49053.0,50580.0,51857.0,71429.0,73332.0,75204.0,75748.0,76769.0,77794.0,78811.0,79331.0,80239.0,81109.0,82294.0,83652.0,85403.0,87137.0,88948.0,90870.0,93091.0,95324.0,98192.0,101927.0,105586.0,109577.0,113702.0,118319.0,125048.0,132758.0,142539.0,153517.0,167515.0,179111.0,191127.0,209839.0,234073.0,266073.0,292142.0,332930.0,372755.0,413467.0,462684.0,509164.0,571659.0,634813.0,693282.0,750890.0,823626.0,896450.0,972303.0
1,Deaths,Globally,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7426.0,7807.0,8778.0,9840.0,11184.0,12783.0,14509.0,16231.0,18433.0,20834.0,23335.0,26493.0,29891.0,33106.0,36405.0,40598.0,45525.0,50321.0
2,Confirmed,China,Western Pacific Region,278.0,309.0,571.0,830.0,1297.0,1985.0,2741.0,4537.0,5997.0,7736.0,9720.0,11821.0,14411.0,17238.0,20471.0,24363.0,28060.0,31211.0,34598.0,37251.0,40235.0,42708.0,44730.0,46550.0,48548.0,50054.0,51174.0,70635.0,72528.0,74280.0,74675.0,75569.0,76392.0,77042.0,77262.0,77780.0,78191.0,78630.0,78961.0,79394.0,79968.0,80174.0,80304.0,80422.0,80565.0,80711.0,80813.0,80859.0,80904.0,80924.0,80955.0,80981.0,80991.0,81021.0,81048.0,81077.0,81116.0,81116.0,81174.0,81300.0,81416.0,81498.0,81601.0,81747.0,81848.0,81961.0,82078.0,82230.0,82341.0,82447.0,82545.0,82631.0,82724.0,82802.0
3,Confirmed,Outside of China,NaN,4.0,5.0,10.0,16.0,23.0,29.0,57.0,56.0,68.0,82.0,106.0,132.0,146.0,153.0,159.0,191.0,216.0,270.0,288.0,307.0,319.0,395.0,441.0,447.0,505.0,526.0,683.0,794.0,804.0,924.0,1073.0,1200.0,1402.0,1769.0,2069.0,2459.0,2918.0,3664.0,4691.0,6009.0,7169.0,8774.0,10566.0,12669.0,14759.0,17481.0,21110.0,24727.0,28673.0,32778.0,37364.0,44067.0,51767.0,61518.0,72469.0,86438.0,97995.0,110011.0,128665.0,152773.0,184657.0,210644.0,251329.0,291008.0,331619.0,380723.0,427086.0,489429.0,552472.0,610835.0,668345.0,740995.0,813726.0,889501.0
4,Deaths,China,Western Pacific Region,NaN,NaN,NaN,NaN,NaN,NaN,80.0,106.0,132.0,170.0,213.0,259.0,304.0,361.0,425.0,491.0,564.0,637.0,723.0,812.0,909.0,1017.0,1114.0,1260.0,1381.0,1524.0,1666.0,1772.0,1870.0,2006.0,2121.0,2239.0,2348.0,2445.0,2595.0,2666.0,2718.0,2747.0,2791.0,2838.0,2873.0,2915.0,2946.0,2984.0,3015.0,3045.0,3073.0,3100.0,3123.0,3140.0,3162.0,3173.0,4955.0,3194.0,3204.0,3218.0,3231.0,3231.0,3242.0,3253.0,3261.0,3267.0,3276.0,3283.0,3287.0,3293.0,3298.0,3301.0,3306.0,3310.0,3314.0,3321.0,3327.0,3331.0
5,Hubei,China,Western Pacific Region,258.0,270.0,375.0,375.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7153.0,9074.0,11177.0,13522.0,16678.0,19665.0,22112.0,24953.0,27100.0,29631.0,31728.0,33366.0,34874.0,51968.0,54406.0,56249.0,58182.0,59989.0,61682.0,62031.0,62662.0,63454.0,64084.0,64287.0,64786.0,65187.0,65596.0,65914.0,66337.0,66907.0,67103.0,67217.0,67332.0,67466.0,67592.0,67666.0,67707.0,67743.0,67760.0,67773.0,67781.0,67786.0,67790.0,67794.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Guangdo

In [0]:
csse_confirmed_cases.loc[csse_confirmed_cases["Country/Region"] == "Germany"]

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20
120,NaN,Germany,51.0,9.0,0,0,0,0,0,1,4,4,4,5,8,10,12,12,12,12,13,13,14,14,16,16,16,16,16,16,16,16,16,16,16,16,16,16,17,27,...,12327,15320,19848,22213,24873,29056,32986,37323,43938,50871,57695,62095,66885,71808,77872,84794,91159,96092,100123,103374,107663,113296,118181,122171,124908,127854,130072,131359,134753,137698,141397,143342,145184,147065,148291,150648,153129,154999,156513,157770


In [0]:
who_conf_and_death_cases.loc[who_conf_and_death_cases["Country/Region"] =="Germany"]

,Province/States,Country/Region,WHO region,1/21/2020,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,2/1/2020,2/2/2020,2/3/2020,2/4/2020,2/5/2020,2/6/2020,2/7/2020,2/8/2020,2/9/2020,2/10/2020,2/11/2020,2/12/2020,2/13/2020,2/14/2020,2/15/2020,2/16/2020,2/17/2020,2/18/2020,2/19/2020,2/20/2020,2/21/2020,2/22/2020,2/23/2020,2/24/2020,2/25/2020,2/26/2020,2/27/2020,2/28/2020,2/29/2020,3/1/2020,3/2/2020,3/3/2020,3/4/2020,3/5/2020,3/6/2020,3/7/2020,3/8/2020,3/9/2020,3/10/2020,3/11/2020,3/12/2020,3/13/2020,3/14/2020,3/15/2020,3/16/2020,3/17/2020,3/18/2020,3/19/2020,3/20/2020,3/21/2020,3/22/2020,3/23/2020,3/24/2020,3/25/2020,3/26/2020,3/27/2020,3/28/2020,3/29/2020,3/30/2020,3/31/2020,4/1/2020,4/2/2020,4/3/2020
61,NaN,Germany,European Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,4.0,4.0,5.0,7.0,8.0,10.0,12.0,12.0,12.0,13.0,14.0,14.0,14.0,14.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,18.0,21.0,26.0,57.0,57.0,129.0,157.0,196.0,262.0,534.0,639.0,795.0,1112.0,1139.0,1296.0,1567.0,2369.0,3062.0,3795.0,4838.0,6012.0,7156.0,8198.0,10999.0,18323.0,21463.0,24774.0,29212.0,31554.0,36508.0,42288.0,48582.0,52547.0,57298.0,61913.0,67366.0,73522.0,79696.0


In [0]:
who_conf_and_death_cases.loc[who_conf_and_death_cases["Country/Region"] =="Madagascar"]

,Province/States,Country/Region,WHO region,1/21/2020,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,2/1/2020,2/2/2020,2/3/2020,2/4/2020,2/5/2020,2/6/2020,2/7/2020,2/8/2020,2/9/2020,2/10/2020,2/11/2020,2/12/2020,2/13/2020,2/14/2020,2/15/2020,2/16/2020,2/17/2020,2/18/2020,2/19/2020,2/20/2020,2/21/2020,2/22/2020,2/23/2020,2/24/2020,2/25/2020,2/26/2020,2/27/2020,2/28/2020,2/29/2020,3/1/2020,3/2/2020,3/3/2020,3/4/2020,3/5/2020,3/6/2020,3/7/2020,3/8/2020,3/9/2020,3/10/2020,3/11/2020,3/12/2020,3/13/2020,3/14/2020,3/15/2020,3/16/2020,3/17/2020,3/18/2020,3/19/2020,3/20/2020,3/21/2020,3/22/2020,3/23/2020,3/24/2020,3/25/2020,3/26/2020,3/27/2020,3/28/2020,3/29/2020,3/30/2020,3/31/2020,4/1/2020,4/2/2020,4/3/2020
213,NaN,Madagascar,African Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,12.0,13.0,19.0,19.0,24.0,26.0,26.0,37.0,46.0,53.0,53.0,65.0


In [0]:
who_conf_and_death_cases[2:4]

,Province/States,Country/Region,WHO region,1/21/2020,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,2/1/2020,2/2/2020,2/3/2020,2/4/2020,2/5/2020,2/6/2020,2/7/2020,2/8/2020,2/9/2020,2/10/2020,2/11/2020,2/12/2020,2/13/2020,2/14/2020,2/15/2020,2/16/2020,2/17/2020,2/18/2020,2/19/2020,2/20/2020,2/21/2020,2/22/2020,2/23/2020,2/24/2020,2/25/2020,2/26/2020,2/27/2020,2/28/2020,2/29/2020,3/1/2020,3/2/2020,3/3/2020,3/4/2020,3/5/2020,3/6/2020,3/7/2020,3/8/2020,3/9/2020,3/10/2020,3/11/2020,3/12/2020,3/13/2020,3/14/2020,3/15/2020,3/16/2020,3/17/2020,3/18/2020,3/19/2020,3/20/2020,3/21/2020,3/22/2020,3/23/2020,3/24/2020,3/25/2020,3/26/2020,3/27/2020,3/28/2020,3/29/2020,3/30/2020,3/31/2020,4/1/2020,4/2/2020,4/3/2020
2,Confirmed,China,Western Pacific Region,278.0,309.0,571.0,830.0,1297.0,1985.0,2741.0,4537.0,5997.0,7736.0,9720.0,11821.0,14411.0,17238.0,20471.0,24363.0,28060.0,31211.0,34598.0,37251.0,40235.0,42708.0,44730.0,46550.0,48548.0,50054.0,51174.0,70635.0,72528.0,74280.0,74675.0,75569.0,76392.0,77042.0,77262.0,77780.0,78191.0,78630.0,78961.0,79394.0,79968.0,80174.0,80304.0,80422.0,80565.0,80711.0,80813.0,80859.0,80904.0,80924.0,80955.0,80981.0,80991.0,81021.0,81048.0,81077.0,81116.0,81116.0,81174.0,81300.0,81416.0,81498.0,81601.0,81747.0,81848.0,81961.0,82078.0,82230.0,82341.0,82447.0,82545.0,82631.0,82724.0,82802.0
3,Confirmed,Outside of China,NaN,4.0,5.0,10.0,16.0,23.0,29.0,57.0,56.0,68.0,82.0,106.0,132.0,146.0,153.0,159.0,191.0,216.0,270.0,288.0,307.0,319.0,395.0,441.0,447.0,505.0,526.0,683.0,794.0,804.0,924.0,1073.0,1200.0,1402.0,1769.0,2069.0,2459.0,2918.0,3664.0,4691.0,6009.0,7169.0,8774.0,10566.0,12669.0,14759.0,17481.0,21110.0,24727.0,28673.0,32778.0,37364.0,44067.0,51767.0,61518.0,72469.0,86438.0,97995.0,110011.0,128665.0,152773.0,184657.0,210644.0,251329.0,291008.0,331619.0,380723.0,427086.0,489429.0,552472.0,610835.0,668345.0,740995.0,813726.0,889501.0


In [0]:
print(who_cases_mdg)


who_cases_mdg.info()


         Total cases  new cases  local transmission  Cluster  sporadic
Date                                                                  
3/21/20            3          3                 NaN      NaN       NaN
3/22/20            3          0                 NaN      NaN       NaN
3/23/20           12          9                 NaN      NaN       NaN
3/24/20           13          1                 NaN      NaN       NaN
3/25/20           19          6                 NaN      NaN       NaN
3/26/20           19          0                 NaN      NaN       NaN
3/27/20           24          5                 NaN      NaN       NaN
3/28/20           26          2                 NaN      NaN       NaN
3/29/20           26          0                 NaN      NaN       NaN
3/30/20           37         11                 NaN      NaN       NaN
3/31/20           46          9                 NaN      NaN       NaN
4/1/20            53          7                 0.0      NaN       NaN
4/2/20

In [0]:
date_data_begin = datetime.date(2020,4,1)
date_data_end = datetime.date(2020,4,15)


format_date = lambda date_py: '{}/{}/{}'.format(date_py.month, date_py.day,
                                                 str(date_py.year)[2:4])

date_formatted_begin = format_date(date_data_begin)
date_formatted_end = format_date(date_data_end)

cases_j_m_obs =  np.array(
    csse_confirmed_cases.loc[csse_confirmed_cases["Country/Region"] == "Madagascar", 
                        date_formatted_begin:date_formatted_end])[0]


cases_w_m_obs =  np.array(who_cases_mdg.loc[date_formatted_begin:date_formatted_end, 'Total cases'])

cases_c_m_obs =  np.array(cco_cases_mdg.loc[date_formatted_begin:date_formatted_end, 'confirmes tvm'])
                      
print('cases who mdg obs:', cases_w_m_obs)

print('cases jhu mdg obs:', cases_j_m_obs)

print('cases cco mdg obs:', cases_c_m_obs)



cases who mdg obs: [ 53  53  65  65  70  77  77  92  93  95  95 104 106 106 110]
cases jhu mdg obs: [ 57  59  70  70  72  82  88  93  93  93 102 106 106 108 110]
cases cco mdg obs: [ 54.  59.  65.  71.  72.  82.  88.  93.  93. 102. 102. 106. 106. 108.
 110.]


In [0]:
date_data_begin = datetime.date(2020,4,2)
date_data_end = datetime.date(2020,4,3)

format_date = lambda date_py: '{}/{}/{}'.format(date_py.month, date_py.day,
                                                 str(date_py.year))

date_formatted_begin = format_date(date_data_begin)
date_formatted_end = format_date(date_data_end)


cases_jw_obs =  who_conf_and_death_cases.loc[(who_conf_and_death_cases['Province/States'] == "Confirmed") & (who_conf_and_death_cases["Country/Region"] == "China"), 
                        date_formatted_begin:date_formatted_end]
print('cases jwho china obs:', cases_jw_obs)

cases_jw_m_obs =  who_conf_and_death_cases.loc[who_conf_and_death_cases["Country/Region"] == "Madagascar", 
                        date_formatted_begin:date_formatted_end]
print('cases jwho md obs:', cases_jw_m_obs)


cases jwho china obs:    4/2/2020  4/3/2020
2   82724.0   82802.0
cases jwho md obs:      4/2/2020  4/3/2020
213      53.0      65.0


In [0]:
#to create a dataframe
df = pd.DataFrame({'A': ['foo', 'bar', 'foo', 'bar',
                            'foo', 'bar', 'foo', 'foo'],
                      'B': ['one', 'one', 'two', 'three',
                            'two', 'two', 'one', 'three'],
                     'C': np.random.randn(8),
                     'D': np.random.randn(8)})
df

,A,B,C,D
0,foo,one,-1.332112,-1.355544
1,bar,one,0.779708,0.776279
2,foo,two,0.216639,-0.891640
3,bar,three,0.120879,0.558460
4,foo,two,0.139912,0.765848
5,bar,two,-0.103522,-0.305206
6,foo,one,0.121314,-2.347776
7,foo,three,0.326310,1.365603


In [0]:
dIvao = pd.DataFrame(data =Ivao)

dIvao['Datint'] = range(len(Ivao))

dIvao['Voatomb'].loc[dIvao.index == datyf_data_manom]
dIvao['Voatomb'].loc[datyf_data_manom]

In [0]:
def delay_cases(new_I_t, len_new_I_t, len_new_cases_obs , delay, delay_arr):
    """
    Delays the input new_I_t by delay and return and array with length len_new_cases_obs
    The initial delay of the output is set by delay_arr. 
    
    Take care that delay is smaller or equal than delay_arr, otherwise zeros are 
    returned, which could potentially lead to errors

    Also assure that len_new_I_t is larger then len(new_cases_obs)-delay, otherwise it 
    means that the simulated data is not long enough to be fitted to the data.
    """

    """
    Mahasedra comments:
    The function delay_cases() constructs the newly reported cases (C_t) of the article (p. 11. Eq. 4).
    Indeed, it will be used below in many equalities of the form: new_cases_inferred = delay_cases(...),
    where: 
    new_cases_inferred is the number of new cases given by the SIR model 
    new_cases_obs is the newly reported or observed cases.

    The function delay_cases() is constructed by building other 2 functions: 
    make_delay_matrix() and interpolate(). [The constructions of these 
    2 functions are just programming details.]
    """



    delay_mat = make_delay_matrix(n_rows=len_new_I_t, 
                                  n_columns=len_new_cases_obs, initial_delay=delay_arr)
    inferred_cases = interpolate(new_I_t, delay, delay_mat)
    return inferred_cases 

def make_delay_matrix(n_rows, n_columns, initial_delay=0):
    """
    Has in each entry the delay between the input with size n_rows and the output
    with size n_columns
    """
    size = max(n_rows, n_columns)
    mat = np.zeros((size, size))
    for i in range(size):
        diagonal = np.ones(size-i)*(initial_delay + i)
        mat += np.diag(diagonal, i)
    for i in range(1, size):
        diagonal = np.ones(size-i)*(initial_delay - i)
        mat += np.diag(diagonal, -i)
    return mat[:n_rows, :n_columns]
    
    """
    Mahasedra comments:
    Above is the construction of make_delay_matrix().
    """

def interpolate(array, delay, delay_matrix):
    interp_matrix = tt.maximum(1-tt.abs_(delay_matrix - delay), 0)
    interpolation = tt.dot(array,interp_matrix)
    return interpolation


ex1 = delay_cases([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], 14, 6, 3, 10)
#ex1
type(ex1)
ex1.shape

ex2 = make_delay_matrix(29,14,16)
# ex2 is the delay_mat for len(new_I_t) = 
print('make_delay_matrix(29,14,16) is:')
ex2

ex3 = np.maximum(1-np.absolute(ex2 - 8),0)
print('interp_matrix for make_delay_matrix(29,14,16) and a delay D = 8 is:')
ex3
# For a discrete time, delay_case built newly reported cases (C_t) from 
# newly infected case at time t - delay.

make_delay_matrix(29,14,16) is:
interp_matrix for make_delay_matrix(29,14,16) and a delay D = 8 is:


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0

# Construction of new_case_inferred

## 1/ Input values for new_case_inferred  
For all cases, the following inputs values of new_case_inferred do not change:  \
delay = delay,  (delay is a random variable) \
delay_arr = diff_data_sim (= 16)

1.1/ new_case_inferred from the training of the model 

In [0]:
    S_begin = N_germany - I_begin
    S_past, I_past, new_I_past = SIR_model(λ=λ * tt.ones(num_days-1), μ=μ, 
                                               S_begin=S_begin, I_begin=I_begin,
                                               N=N_germany)
    new_cases_obs = np.diff(cases_obs)
    new_cases_inferred = delay_cases(new_I_past, len_new_I_t=num_days - 1, 
                                     len_new_cases_obs = len(new_cases_obs), 
                                     delay = delay, 
                                     delay_arr = diff_data_sim
                                     )

For the training of the model, the input values for new_cases_inferred are: \
    new_I_t = new_I_past, \
    len_new_I_t = num_days - 1 = 29 \
    len_new_cases_obs = len(new_cases_obs) = len(cases_obs) - 1 = 15 - 1 = 14

1.2/ Prediction \
For all cases of prediction, the following input values are fixed: \
len_new_I_t = diff_data_sim + num_days_to_predict = 16 + 28 = 44,     
len_new_cases_obs = num_days_to_predict = 28 \

For each case of prediction, the explicit construction of new_I_t changes.

In [0]:
# prediction, start with no changes in policy
    S_begin = S_past[-1]
    I_begin = I_past[-1]
    forecast_no_change = SIR_model(λ=λ*tt.ones(num_days_to_predict), μ=μ, 
                        S_begin=S_begin, I_begin=I_begin, N=N_germany)
    S_no_change, I_no_change, new_I_no_change = forecast_no_change

new_cases_inferred = delay_cases(tt.concatenate([new_I_past[-diff_data_sim:], new_I_no_change]),
                                len_new_I_t = diff_data_sim + num_days_to_predict, 
                                len_new_cases_obs = num_days_to_predict, 
                                delay = delay, # delay is a random variable
                                delay_arr = diff_data_sim
 # One preserves the last 16 (= diff_data_sim) elements of new_I_past 
 # and concatenates them with the 28 (= num_days_to_predict) elements of new_I_no_change

In [0]:
 # social distancing, m reduced by about 50 percent

new_cases_inferred = delay_cases(tt.concatenate([new_I_past[-diff_data_sim:], new_I_soc_dist]), 
                                len_new_I_t = diff_data_sim + num_days_to_predict, 
                                len_new_cases_obs = num_days_to_predict, 
                                delay = delay, 
                                delay_arr = diff_data_sim
                                 )
                               

In [0]:
# isolation, almost no new infections besides baseline after transient phase

new_cases_inferred = delay_cases(tt.concatenate([new_I_past[-diff_data_sim:], new_I_isol]), 
                                len_new_I_t = diff_data_sim + num_days_to_predict, 
                                len_new_cases_obs = num_days_to_predict, 
                                delay = delay, 
                                delay_arr = diff_data_sim
                                 )
                                 
                              

In [0]:
 # isolation 5 days later, almost no new infections besides baseline after transient phase
 new_cases_inferred = delay_cases(tt.concatenate([new_I_past[-diff_data_sim:], new_I_isol_later]), 
                                  len_new_I_t = diff_data_sim + num_days_to_predict, 
                                  len_new_cases_obs = num_days_to_predict, 
                                  delay = delay, delay_arr = diff_data_sim
                                  )

In [0]:
# isolation 5 days earlier, almost no new infections besides baseline after transient phase
# length_transient = 7

new_cases_inferred = delay_cases(tt.concatenate([new_I_past[-diff_data_sim:days_offset], new_I_earlier]), 
                                 len_new_I_t=diff_data_sim + num_days_to_predict, 
                                 len_new_cases_obs=num_days_to_predict, 
                                delay=delay, delay_arr=diff_data_sim
                                 )

In [0]:
# long transient scenario
#  length_transient = 14
new_cases_inferred = delay_cases(tt.concatenate([new_I_past[-diff_data_sim:days_offset_sim], new_I_long_trans]), 
                                 len_new_I_t=diff_data_sim + num_days_to_predict, 
                                 len_new_cases_obs=num_days_to_predict, 
                                 delay=delay, delay_arr=diff_data_sim
                                 )

In [0]:
# immediate transient scenario
new_cases_inferred = delay_cases(tt.concatenate([new_I_past[-diff_data_sim:], new_I_isol]), 
                                 len_new_I_t=diff_data_sim + num_days_to_predict, 
                                 len_new_cases_obs=num_days_to_predict, 
                                 delay=delay, delay_arr=diff_data_sim
                                 )

In [0]:
    # -------------------------------------------------------------------------- #
    # Data for the training of the model 
    # -------------------------------------------------------------------------- #

    
    # For the training: